In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# Khởi động driver chorm và get url

In [2]:
driver = webdriver.Chrome("C:\chromedriver_win32\chromedriver.exe")
driver.maximize_window()
url = 'https://tiki.vn/dien-thoai-may-tinh-bang/c1789'
driver.implicitly_wait(30)
driver.get(url)

C:\Users\DELL\AppData\Local\Temp\ipykernel_1716\372970283.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\chromedriver_win32\chromedriver.exe")


# Lấy Link của các sản phẩm page 1

In [3]:
page_source = BeautifulSoup(driver.page_source)
product_item =  page_source.find_all('a', class_='product-item')
links = [link.get('href') for link in product_item]
len(links)

40

# Lấy tên sản phẩm

In [4]:
name_products= page_source.find_all('div', class_='name')
name_products = [name.text for name in name_products]
len(name_products)

40

# Lấy giá bán sản phẩm

In [5]:
price_products = page_source.find_all('div', class_='price-discount__price')
price_products = [price.text for price in price_products]
len(price_products)

40

# Lấy sao đánh giá và số lượng bán ra sản phẩm

In [6]:
info = page_source.find_all('div', 'info')
rate_products = []
sold_out = []
for inf in info:
    if inf.find(class_='total') == None:
        rate_products.append(0)
    else: rate_products.append(inf.find(class_='total').text)
    
    if inf.find(class_='styles__StyledQtySold-sc-732h27-2 fCfYNm') == None:
        sold_out.append(0)
    else: sold_out.append(inf.find(class_='styles__StyledQtySold-sc-732h27-2 fCfYNm').text)

print(len(rate_products))
print(len(sold_out))

40
40


# Lấy comment của mỗi sản phẩm

In [7]:
for i in range(len(links)):
    # tiền xử lý link
    if links[i][:2] == '//':
        links[i] = 'https:' + links[i]
    else: links[i] = 'https://tiki.vn' + links[i]

In [8]:
links

['https://tka.tiki.vn/pixel/pixel?data=ZiBvMHw7suHlHbPaTWMdTKO59zeZNXGt2V1M0-JeaIUZ_VxhpJaDRhpHkmHSPCH6NGGgV0TNJumHP6Uhm5sCdj16sxeH4XtfxJJ6WP3nV_3GD4qghM6bbX2V9Qzow2N0KPokgTsOHs7t-506-Fz088VJsLZHpjMQAxtnkfu2Z_BUSz-gf5Nga-dW10AfXTAZg1hKnVHAdcH4_Jh8ROCQhzqX5_eheDa3WxiPl1ZzBR7HZJtJVovsTfjnTB-_IRf1bma6QVW18Fl6RDyU5lKF6hub1RhZBg2VTHHCtUKXDiEmbFi6VKZUxTO1IpwfiK3sKVKx_XDIKY2V3FF83hgVp8bGO4evAQOCeYa9eGkOROc2luRnPra9y54jdNruAS-uTpY1huxRbMqjPouYTDuWtzgeiHsSGP5LI8iZJM3vEHtN-fLGej_bNqa5gHh1bR6MuJSITV7rJcweM0bmRSFw8hM3lH2tVyU66IIWSLQkxb5bShx431DRSRxlgbykWA1zveRL0_QcWB1TzQkKU8k0smlIfL-uSKv2_sbpvEP70V9WufacemltmSg1J3kxs5U2w2-IGmMm22-cAUVXakJ4C_lox7ShUnCNeh_ixRnr_lheM8k5xTc2wtO_hC3eY3b19XNHNSMHmWNmq-wwgC2067YPp1ogxyDkHFsRYnmN_dpKRH5WpNyZWxwtHRB7absjj5zcB28ImVoQPBfGiL8llJal-fu66i2B2zGI82J6wGzbSL9EUjbRnNWvOw2ENkAqjnf2qDoi4Nzc2bxo5kM29ndlAy11jLkD2JTVfQxNB_hltdD-dzlE1IIAKL5lGpWTaVzSOTyS8ouGO3dAd0zcIjqg682HUCEC7-U2bMM1d-iGkDe9PgJcrZ6Xq7emCbD9q4AxdodPo0RMmQEHQ5vlCM2coLsn5fwVNXecfJpM1roCShx8tf08VPqg0pHcCVcVV

In [9]:
import time
comments = []
for i in range(len(links)):
    print(f'Crawling Link {i+1}:')
    driver.get(links[i])
    
    page_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight-3000);")
    time.sleep(4)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight-2000);")
    time.sleep(4)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight-1000);")

    cmts, number_tab = [], 1
    while True:
        page_source = BeautifulSoup(driver.page_source)
        reviews = page_source.find_all('div', class_='review-comment__content')
        reviews = [review.text for review in reviews]
        for idx in range(len(reviews)):
            if reviews[idx] != []:
                cmts.append(reviews[idx])
        if len(cmts) >= 50:
            break
        try:
            print(f'    Crawling tab{number_tab}')  
            next_cmts = driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/main/div[3]/div[4]/div/div[2]/div[8]/ul/li[7]/a')
            next_cmts.click()
            number_tab += 1                     
        except: break
        
    comments.append(cmts)
    
    

Crawling Link 1:
    Crawling tab1
    Crawling tab2
    Crawling tab3
    Crawling tab4
    Crawling tab5
    Crawling tab6
    Crawling tab7
Crawling Link 2:
    Crawling tab1
    Crawling tab2
    Crawling tab3
    Crawling tab4
    Crawling tab5
    Crawling tab6
    Crawling tab7
    Crawling tab8
    Crawling tab9
Crawling Link 3:
    Crawling tab1
    Crawling tab2
    Crawling tab3
    Crawling tab4
    Crawling tab5
    Crawling tab6
    Crawling tab7
    Crawling tab8
    Crawling tab9
Crawling Link 4:
    Crawling tab1
Crawling Link 5:
    Crawling tab1
Crawling Link 6:
    Crawling tab1
Crawling Link 7:
    Crawling tab1
Crawling Link 8:
    Crawling tab1
Crawling Link 9:
    Crawling tab1
    Crawling tab2
    Crawling tab3
    Crawling tab4
    Crawling tab5
    Crawling tab6
    Crawling tab7
    Crawling tab8
    Crawling tab9
Crawling Link 10:
    Crawling tab1
    Crawling tab2
Crawling Link 11:
    Crawling tab1
    Crawling tab2
    Crawling tab3
    Crawling tab4
 

In [ ]:
comments

[['Máy nguyên seal, cơ mà bên vận chuyển nhấn thành đã giao hàng làm tưởng bên họ có người lấy hàng của tôi. Dù vậy vẫn cho 5 sao vì máy vẫn còn nguyên, chưa khui seal',
  'Điện thoại đẹp, đáp ứng đủ nhu cầu cần thiết, màn hình full HD+, pin trâu, có hỗ trợ sạc nhanh, quá ok với tầm giá\nTiki giao hàng siêu nhanh, có kèm mã bảo mật khi nhận hàng đã thanh toán giá trị cao\nCho 5 sao',
  'Hàng nguyên seal, chất lượng, giao hàng nhanh.',
  'Hàng giao nhanh, chuẩn, được đóng hộp rất kĩ càng. Sản phẩm vượt ngoài sự mong đợi, rất cảm ơn Tiki, sẽ ủng hộ Tiki nhiều hơn nữa',
  'Giao hàng nhanh, đóng gói kỹ, hài lòng!',
  'Máy nguyên seal, cơ mà bên vận chuyển nhấn thành đã giao hàng làm tưởng bên họ có người lấy hàng của tôi. Dù vậy vẫn cho 5 sao vì máy vẫn còn nguyên, chưa khui seal',
  'Điện thoại đẹp, đáp ứng đủ nhu cầu cần thiết, màn hình full HD+, pin trâu, có hỗ trợ sạc nhanh, quá ok với tầm giá\nTiki giao hàng siêu nhanh, có kèm mã bảo mật khi nhận hàng đã thanh toán giá trị cao\nCho 5 